#   <center> <span style="color:blue"> Azure Linux VM Size and Price at Glance </span> </center>

##   <span style="color:blue"> Overview </span>
<span  STYLE="color: black font-size: 12pt"> This notebook was developped to assist a customer who needed to have the VM Sizes and Prices for all available Azure VM shapes in one place to assist with database worload sizing.  </span>
The key metrics for proper database workload sizing include: vCPU, memory, Max IOPS, Max IO Throughput MBPS, monthly cost, Max Data disks etc.  
Typical database workloads on IaaS VM are usually IO bound instead of CPU or Memory at the VM level. So it's critical to size properly based on the application workload IOPS and IO throughput MBPS. Azure VM size and price information can be found in https://docs.microsoft.com/en-us/azure/virtual-machines/sizes and https://azure.microsoft.com/en-us/pricing/details/virtual-machines/linux/; However the information is scattered in multiple linked pages which made it difficult to navigate and compare. The goal of this notebook is to combine all these information in one spreadsheet for easy comparison and decision making.  
We use BeautifulSoup to scrap the information from Microsoft document site and also https://azureprice.net/. The final product is an excel sheet with key metrics for all available Azure Linux VMs. The code would work for future released VM size as well as long as the embeded size table in the microsoft VM size page has the same/similar column formats. 


In [74]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import urlparse, parse_qs
import re, os, io, zipfile
import json
import requests
from requests import exceptions
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from urllib.parse  import  urljoin
 

##  
<span  STYLE="color: black font-size: 15pt"> For convenience, We extract VM price from azureprice.net which seems to be maintained reguarly. We use the west region for our price reference. Estimated monthly cost was computed. </span>

In [75]:
def requests_retry_session(retries=3, 
                           backoff_factor=0.3, 
                           status_forcelist=(500, 502, 503, 504), 
                           session=None):
    '''
    Use/Create an http(s) requests session that will retry a request.
    '''
    session = session or requests.Session()
    retry = Retry(total = retries, 
                  read = retries, 
                  connect = retries, 
                  backoff_factor = backoff_factor, 
                  status_forcelist = status_forcelist)
    adapter = HTTPAdapter(max_retries = retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    return session

    
def get_pricing_json(pricing_url):

        try:
            
            r = requests_retry_session().get(pricing_url, timeout = 300)
        except Exception as e:
            
            jsonfile = []
            exit
        r_content = r.content
         
         
        soup = BeautifulSoup(r_content, "html.parser")
        found=0
        for script in soup.find_all('script'):
           #print(i)
           mystr=' '.join(script.contents)
           if 'json =' in mystr:
               return mystr 
                

In [76]:
pricing_json=get_pricing_json('https://azureprice.net/')[13:-5] 
print(pricing_json)

[{"name":"Standard_E96as_v4","canonicalname":"e96as_v4","numberOfCores":96,"osDiskSizeInMB":1047552,"resourceDiskSizeInMB":1376256,"memoryInMB":688128,"maxDataDiskCount":32,"regionId":"westus","regionName":"West US","currency":"USD","isTaxIncluded":false,"isLowPriority":false,"linuxPrice":6.72,"windowsPrice":11.136,"supportPremiumDisk":true,"pricePerMemoryLinux":0.01,"pricePerMemoryWindows":0.0166,"pricePerCoreLinux":0.07,"pricePerCoreWindows":0.116,"bestPriceRegion":"Central India / -43.3","bestSecondPriceRegion":"East US / -10","modifiedDate":"2020-11-15 10:01:12Z"},{"name":"Standard_E96-48as_v4","canonicalname":"e96-48as_v4","numberOfCores":96,"osDiskSizeInMB":1047552,"resourceDiskSizeInMB":1376256,"memoryInMB":688128,"maxDataDiskCount":32,"regionId":"westus","regionName":"West US","currency":"USD","isTaxIncluded":false,"isLowPriority":false,"linuxPrice":6.72,"windowsPrice":11.136,"supportPremiumDisk":true,"pricePerMemoryLinux":0.01,"pricePerMemoryWindows":0.0166,"pricePerCoreLinux"

In [77]:
jdata = json.loads(pricing_json)
for d in jdata:
    for key, value in d.items():
        print(key, value)

name Standard_E96as_v4
canonicalname e96as_v4
numberOfCores 96
osDiskSizeInMB 1047552
resourceDiskSizeInMB 1376256
memoryInMB 688128
maxDataDiskCount 32
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 6.72
windowsPrice 11.136
supportPremiumDisk True
pricePerMemoryLinux 0.01
pricePerMemoryWindows 0.0166
pricePerCoreLinux 0.07
pricePerCoreWindows 0.116
bestPriceRegion Central India / -43.3
bestSecondPriceRegion East US / -10
modifiedDate 2020-11-15 10:01:12Z
name Standard_E96-48as_v4
canonicalname e96-48as_v4
numberOfCores 96
osDiskSizeInMB 1047552
resourceDiskSizeInMB 1376256
memoryInMB 688128
maxDataDiskCount 32
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 6.72
windowsPrice 11.136
supportPremiumDisk True
pricePerMemoryLinux 0.01
pricePerMemoryWindows 0.0166
pricePerCoreLinux 0.07
pricePerCoreWindows 0.116
bestPriceRegion East US / -10
bestSecondPriceRegion East US 2 / -10
modifiedDa

numberOfCores 24
osDiskSizeInMB 1047552
resourceDiskSizeInMB 688128
memoryInMB 229376
maxDataDiskCount 24
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 2.28
windowsPrice 3.384
supportPremiumDisk True
pricePerMemoryLinux 0.0102
pricePerMemoryWindows 0.0151
pricePerCoreLinux 0.095
pricePerCoreWindows 0.141
bestPriceRegion East US / 0
bestSecondPriceRegion East US 2 / 0
modifiedDate 2020-11-15 10:01:12Z
name Standard_NV12s_v3
canonicalname nv12s_v3
numberOfCores 12
osDiskSizeInMB 1047552
resourceDiskSizeInMB 344064
memoryInMB 114688
maxDataDiskCount 12
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 1.14
windowsPrice 1.692
supportPremiumDisk True
pricePerMemoryLinux 0.0102
pricePerMemoryWindows 0.0151
pricePerCoreLinux 0.095
pricePerCoreWindows 0.141
bestPriceRegion East US / 0
bestSecondPriceRegion East US 2 / 0
modifiedDate 2020-11-15 10:01:12Z
name Standard_NV24s_v2
canonicalname nv2

numberOfCores 8
osDiskSizeInMB 1047552
resourceDiskSizeInMB 0
memoryInMB 32768
maxDataDiskCount 16
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 0.448
windowsPrice 0.816
supportPremiumDisk False
pricePerMemoryLinux 0.014
pricePerMemoryWindows 0.0255
pricePerCoreLinux 0.056
pricePerCoreWindows 0.102
bestPriceRegion East US / -14.3
bestSecondPriceRegion East US 2 / -14.3
modifiedDate 2020-11-15 10:01:12Z
name Standard_D4_v4
canonicalname d4_v4
numberOfCores 4
osDiskSizeInMB 1047552
resourceDiskSizeInMB 0
memoryInMB 16384
maxDataDiskCount 8
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 0.224
windowsPrice 0.408
supportPremiumDisk False
pricePerMemoryLinux 0.014
pricePerMemoryWindows 0.0255
pricePerCoreLinux 0.056
pricePerCoreWindows 0.102
bestPriceRegion East US / -14.3
bestSecondPriceRegion East US 2 / -14.3
modifiedDate 2020-11-15 10:01:12Z
name Standard_D2_v4
canonicalname d2_v4
num

windowsPrice 2.032
supportPremiumDisk True
pricePerMemoryLinux 0.0101
pricePerMemoryWindows 0.0159
pricePerCoreLinux 0.081
pricePerCoreWindows 0.127
bestPriceRegion East US / -11.1
bestSecondPriceRegion East US 2 / -11.1
modifiedDate 2020-11-15 10:01:12Z
name Standard_E16-4ds_v4
canonicalname e16-4ds_v4
numberOfCores 16
osDiskSizeInMB 1047552
resourceDiskSizeInMB 614400
memoryInMB 131072
maxDataDiskCount 32
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 1.296
windowsPrice 2.032
supportPremiumDisk True
pricePerMemoryLinux 0.0101
pricePerMemoryWindows 0.0159
pricePerCoreLinux 0.081
pricePerCoreWindows 0.127
bestPriceRegion East US / -11.1
bestSecondPriceRegion East US 2 / -11.1
modifiedDate 2020-11-15 10:01:12Z
name Standard_E8ds_v4
canonicalname e8ds_v4
numberOfCores 8
osDiskSizeInMB 1047552
resourceDiskSizeInMB 307200
memoryInMB 65536
maxDataDiskCount 16
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority

pricePerCoreLinux 0.063
pricePerCoreWindows 0.109
bestPriceRegion East US / -10
bestSecondPriceRegion East US 2 / -10
modifiedDate 2020-11-15 10:01:12Z
name Standard_E64is_v3
canonicalname e64is_v3
numberOfCores 64
osDiskSizeInMB 1047552
resourceDiskSizeInMB 884736
memoryInMB 442368
maxDataDiskCount 32
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 4.032
windowsPrice 6.976
supportPremiumDisk True
pricePerMemoryLinux 0.0093
pricePerMemoryWindows 0.0161
pricePerCoreLinux 0.063
pricePerCoreWindows 0.109
bestPriceRegion East US / -10
bestSecondPriceRegion East US 2 / -10
modifiedDate 2020-11-15 10:01:12Z
name Standard_E48s_v3
canonicalname e48s_v3
numberOfCores 48
osDiskSizeInMB 1047552
resourceDiskSizeInMB 786432
memoryInMB 393216
maxDataDiskCount 32
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 3.36
windowsPrice 5.568
supportPremiumDisk True
pricePerMemoryLinux 0.0087
pricePerMemoryWi

isLowPriority False
linuxPrice 0.936
windowsPrice 1.672
supportPremiumDisk False
pricePerMemoryLinux 0.0146
pricePerMemoryWindows 0.0261
pricePerCoreLinux 0.0585
pricePerCoreWindows 0.1045
bestPriceRegion East US / -17.9
bestSecondPriceRegion East US 2 / -17.9
modifiedDate 2020-11-15 10:01:12Z
name Standard_D8_v3
canonicalname d8_v3
numberOfCores 8
osDiskSizeInMB 1047552
resourceDiskSizeInMB 204800
memoryInMB 32768
maxDataDiskCount 16
regionId westus
regionName West US
currency USD
isTaxIncluded False
isLowPriority False
linuxPrice 0.468
windowsPrice 0.836
supportPremiumDisk False
pricePerMemoryLinux 0.0146
pricePerMemoryWindows 0.0261
pricePerCoreLinux 0.0585
pricePerCoreWindows 0.1045
bestPriceRegion East US / -17.9
bestSecondPriceRegion East US 2 / -17.9
modifiedDate 2020-11-15 10:01:12Z
name Standard_D4_v3
canonicalname d4_v3
numberOfCores 4
osDiskSizeInMB 1047552
resourceDiskSizeInMB 102400
memoryInMB 16384
maxDataDiskCount 8
regionId westus
regionName West US
currency USD
isTaxIn

pricePerMemoryLinux 0.0124
pricePerMemoryWindows 0.0216
pricePerCoreLinux 0.0062
pricePerCoreWindows 0.0108
bestPriceRegion East US / -16.1
bestSecondPriceRegion East US 2 / -16.1
modifiedDate 2020-11-15 10:01:12Z


In [78]:
 
price = pd.DataFrame.from_dict(jdata, orient='columns')


In [79]:
price.head(10)

,name,canonicalname,numberOfCores,osDiskSizeInMB,resourceDiskSizeInMB,memoryInMB,maxDataDiskCount,regionId,regionName,currency,...,linuxPrice,windowsPrice,supportPremiumDisk,pricePerMemoryLinux,pricePerMemoryWindows,pricePerCoreLinux,pricePerCoreWindows,bestPriceRegion,bestSecondPriceRegion,modifiedDate
0,Standard_E96as_v4,e96as_v4,96,1047552,1376256,688128,32,westus,West US,USD,...,6.72,11.136,True,0.0100,0.0166,0.07,0.116,Central India / -43.3,East US / -10,2020-11-15 10:01:12Z
1,Standard_E96-48as_v4,e96-48as_v4,96,1047552,1376256,688128,32,westus,West US,USD,...,6.72,11.136,True,0.0100,0.0166,0.07,0.116,East US / -10,East US 2 / -10,2020-11-15 10:01:12Z
2,Standard_E96-24as_v4,e96-24as_v4,96,1047552,1376256,688128,32,westus,West US,USD,...,6.72,11.136,True,0.0100,0.0166,0.07,0.116,East US / -10,East US 2 / -10,2020-11-15 10:01:12Z
3,Standard_E64as_v4,e64as_v4,64,1047552,884736,524288,32,westus,West US,USD,...,4.48,7.424,True,0.0088,0.0145,0.07,0.116,Central India / -43.3,East US / -10,2020-11-15 10:01:12Z
4,Standard_E64-32as_v4,e64-32as_v4,64,1047552,884736,524288,32,westus,West US,USD,...,4.48,7.424,True,0.0088,0.0145,0.07,0.116,East US / -10,East US 2 / -10,2020-11-15 10:01:12Z
5,Standard_E64-16as_v4,e64-16as_v4,64,1047552,884736,524288,32,westus,West US,USD,...,4.48,7.424,True,0.0088,0.0145,0.07,0.116,East US / -10,East US 2 / -10,2020-11-15 10:01:12Z
6,Standard_E48as_v4,e48as_v4,48,1047552,786432,393216,32,westus,West US,USD,...,3.36,5.568,True,0.0087,0.0145,0.07,0.116,Central India / -43.3,East US / -10,2020-11-15 10:01:12Z
7,Standard_E32as_v4,e32as_v4,32,1047552,524288,262144,32,westus,West US,USD,...,2.24,3.712,True,0.0088,0.0145,0.07,0.116,Central India / -43.3,East US / -10,2020-11-15 10:01:12Z
8,Standard_E32-16as_v4,e32-16as_v4,32,1047552,524288,262144,32,westus,West US,USD,...,2.24,3.712,True,0.0088,0.0145,0.07,0.116,East US / -10,East US 2 / -10,2020-11-15 10:01:12Z
9,Standard_E32-8as_v4,e32-8as_v4,32,1047552,524288,262144,32,westus,West US,USD,...,2.24,3.712,True,0.0088,0.0145,0.07,0.116,East US / -10,East US 2 / -10,2020-11-15 10:01:12Z


##  
<span  STYLE="color: black font-size: 15pt"> Below is to parse and format the json file we scrap from the embedded VM size table. </span>

In [80]:
def printsize(size_string):
    try:
        size_1=(re.split(r'_v',size_string))[0]
        size_2=(re.split(r'_v',size_string))[1][0]
        size=size_1 + '_v' + size_2
    except:
        size=size_string
    return size

def specs_parse(url):

    #dfs = pd.read_html('https://docs.microsoft.com/en-us/azure/virtual-machines/ev3-esv3-series?toc=/azure/virtual-machines/linux/toc.json&bc=/azure/virtual-machines/linux/breadcrumb/toc.json',header=0)
    dfs =pd.read_html(url,header=0)
    df_norm_concat=pd.DataFrame()
    for df in dfs:
        #print(df.head(10))
        #print(df.Size)
        df_norm=pd.DataFrame() 
        for cols in df.columns.values:
            if cols == 'Size':
                df_norm['Size']=df[cols].apply(lambda x: printsize(x)).apply(lambda x:x.split(' ')[0])
            if 'CPU' in cols:
                df_norm['VCPU']=df[cols] 
            if 'memory' in cols.lower():
                df_norm['Memory']=df[cols]
            if 'max data' in cols.lower():
                df_norm['Max data disks']=df[cols]
            if 'storage (ssd)' in cols.lower():
                df_norm['Temporary Storage SSD(GB)']=df[cols]
            if 'max cached'in cols.lower():
                df_norm['Max cached IO throughput: IOPS/MBps (cache size in GiB)']=df[cols]
            if 'max uncached'in cols.lower() or 'max burst uncached' in cols.lower():  
                df_norm['Max uncached IO throughput: IOPS/MBps']=df[cols]
            if 'network bandwidth' in cols.lower():
                df_norm['Network bandwidth']=pd.to_numeric(df[cols].astype(str).apply(lambda x:x.split(' ')[0]),errors='coerce')
            if 'Max data disks/throughput: IOPS' in cols: 
                df_norm['Max data disks']=pd.to_numeric(df[cols].apply(lambda x:x.split('/')[0]),errors='coerce')
                df_norm['Max IO throughput']=df[cols].apply(lambda x:x.split('/')[1])
        df_norm_concat=df_norm_concat.append(df_norm ) 
    return(df_norm_concat) 

In [81]:
specs_parse('https://docs.microsoft.com/en-us/azure/virtual-machines/fsv2-series?toc=/azure/virtual-machines/linux/toc.json&bc=/azure/virtual-machines/linux/breadcrumb/toc.json')

,Size,VCPU,Memory,Temporary Storage SSD(GB),Max data disks,Max cached IO throughput: IOPS/MBps (cache size in GiB),Max uncached IO throughput: IOPS/MBps,Network bandwidth
0,Standard_F2s_v2,2,4,16,4,4000/31 (32),3200/47,875
1,Standard_F4s_v2,4,8,32,8,8000/63 (64),6400/95,1750
2,Standard_F8s_v2,8,16,64,16,16000/127 (128),12800/190,3500
3,Standard_F16s_v2,16,32,128,32,32000/255 (256),25600/380,7000
4,Standard_F32s_v2,32,64,256,32,64000/512 (512),51200/750,14000
5,Standard_F48s_v2,48,96,384,32,96000/768 (768),76800/1100,21000
6,Standard_F64s_v2,64,128,512,32,128000/1024 (1024),80000/1100,28000
7,Standard_F72s_v2,72,144,576,32,144000/1152 (1520),80000/1100,30000


In [82]:
specs_parse('https://docs.microsoft.com/en-us/azure/virtual-machines/dv2-dsv2-series-memory?toc=/azure/virtual-machines/linux/toc.json&bc=/azure/virtual-machines/linux/breadcrumb/toc.json')

,Size,VCPU,Memory,Temporary Storage SSD(GB),Max data disks,Max IO throughput,Network bandwidth,Max cached IO throughput: IOPS/MBps (cache size in GiB),Max uncached IO throughput: IOPS/MBps
0,Standard_D11_v2,2,14,100,8,8x500,1500,NaN,NaN
1,Standard_D12_v2,4,28,200,16,16x500,3000,NaN,NaN
2,Standard_D13_v2,8,56,400,32,32x500,6000,NaN,NaN
3,Standard_D14_v2,16,112,800,64,64x500,12000,NaN,NaN
4,Standard_D15_v2,20,140,1000,64,64x500,25000,NaN,NaN
0,Standard_DS11_v2,2,14,28,8,NaN,1500,8000/64 (72),6400/96
1,Standard_DS12_v2,4,28,56,16,NaN,3000,16000/128 (144),12800/192
2,Standard_DS13_v2,8,56,112,32,NaN,6000,32000/256 (288),25600/384
3,Standard_DS14_v2,16,112,224,64,NaN,12000,64000/512 (576),51200/768
4,Standard_DS15_v2,20,140,280,64,NaN,25000,80000/640 (720),64000/960


In [83]:
specs_parse('https://docs.microsoft.com/en-us/azure/virtual-machines/dv3-dsv3-series?toc=/azure/virtual-machines/linux/toc.json&bc=/azure/virtual-machines/linux/breadcrumb/toc.json')

,Size,VCPU,Memory,Temporary Storage SSD(GB),Max data disks,Network bandwidth,Max cached IO throughput: IOPS/MBps (cache size in GiB),Max uncached IO throughput: IOPS/MBps
0,Standard_D2_v3,2,8,50,4,NaN,NaN,NaN
1,Standard_D4_v3,4,16,100,8,NaN,NaN,NaN
2,Standard_D8_v3,8,32,200,16,NaN,NaN,NaN
3,Standard_D16_v3,16,64,400,32,NaN,NaN,NaN
4,Standard_D32_v3,32,128,800,32,NaN,NaN,NaN
5,Standard_D48_v3,48,192,1200,32,NaN,NaN,NaN
6,Standard_D64_v3,64,256,1600,32,NaN,NaN,NaN
0,Standard_D2s_v3,2,8,16,4,NaN,4000/32 (50),4000/100
1,Standard_D4s_v3,4,16,32,8,NaN,8000/64 (100),8000/200
2,Standard_D8s_v3,8,32,64,16,NaN,16000/128 (200),16000/400


In [84]:
specs_parse('https://docs.microsoft.com/en-us/azure/virtual-machines/ev3-esv3-series?toc=/azure/virtual-machines/linux/toc.json&bc=/azure/virtual-machines/linux/breadcrumb/toc.json')

,Size,VCPU,Memory,Temporary Storage SSD(GB),Max data disks,Network bandwidth,Max cached IO throughput: IOPS/MBps (cache size in GiB),Max uncached IO throughput: IOPS/MBps
0,Standard_E2_v3,2,16,50,4,NaN,NaN,NaN
1,Standard_E4_v3,4,32,100,8,NaN,NaN,NaN
2,Standard_E8_v3,8,64,200,16,NaN,NaN,NaN
3,Standard_E16_v3,16,128,400,32,NaN,NaN,NaN
4,Standard_E20_v3,20,160,500,32,NaN,NaN,NaN
5,Standard_E32_v3,32,256,800,32,NaN,NaN,NaN
6,Standard_E48_v3,48,384,1200,32,NaN,NaN,NaN
7,Standard_E64_v3,64,432,1600,32,NaN,NaN,NaN
8,Standard_E64i_v3,64,432,1600,32,NaN,NaN,NaN
0,Standard_E2s_v3,2,16,32,4,NaN,4000/32 (50),3200/48


In [85]:
specs_parse('https://docs.microsoft.com/en-us/azure/virtual-machines/edv4-edsv4-series?toc=/azure/virtual-machines/linux/toc.json&bc=/azure/virtual-machines/linux/breadcrumb/toc.json')

,Size,VCPU,Memory,Temporary Storage SSD(GB),Max data disks,Max cached IO throughput: IOPS/MBps (cache size in GiB),Network bandwidth,Max uncached IO throughput: IOPS/MBps
0,Standard_E2d_v4,2,16,75,4,19000/120,1000,NaN
1,Standard_E4d_v4,4,32,150,8,38500/242,2000,NaN
2,Standard_E8d_v4,8,64,300,16,77000/485,4000,NaN
3,Standard_E16d_v4,16,128,600,32,154000/968,8000,NaN
4,Standard_E20d_v4,20,160,750,32,193000/1211,10000,NaN
5,Standard_E32d_v4,32,256,1200,32,308000/1936,16000,NaN
6,Standard_E48d_v4,48,384,1800,32,462000/2904,24000,NaN
7,Standard_E64d_v4,64,504,2400,32,615000/3872,30000,NaN
0,Standard_E2ds_v4,2,16,75,4,19000/120(50),1000,3200/48
1,Standard_E4ds_v4,4,32,150,8,38500/242(100),2000,6400/96


In [86]:
specs_parse('https://docs.microsoft.com/en-us/azure/virtual-machines/fsv2-series?toc=/azure/virtual-machines/linux/toc.json&bc=/azure/virtual-machines/linux/breadcrumb/toc.json')

,Size,VCPU,Memory,Temporary Storage SSD(GB),Max data disks,Max cached IO throughput: IOPS/MBps (cache size in GiB),Max uncached IO throughput: IOPS/MBps,Network bandwidth
0,Standard_F2s_v2,2,4,16,4,4000/31 (32),3200/47,875
1,Standard_F4s_v2,4,8,32,8,8000/63 (64),6400/95,1750
2,Standard_F8s_v2,8,16,64,16,16000/127 (128),12800/190,3500
3,Standard_F16s_v2,16,32,128,32,32000/255 (256),25600/380,7000
4,Standard_F32s_v2,32,64,256,32,64000/512 (512),51200/750,14000
5,Standard_F48s_v2,48,96,384,32,96000/768 (768),76800/1100,21000
6,Standard_F64s_v2,64,128,512,32,128000/1024 (1024),80000/1100,28000
7,Standard_F72s_v2,72,144,576,32,144000/1152 (1520),80000/1100,30000


In [87]:
 specs_parse('https://docs.microsoft.com/en-us/azure/virtual-machines/dv2-dsv2-series?toc=/azure/virtual-machines/linux/toc.json&bc=/azure/virtual-machines/linux/breadcrumb/toc.json')

,Size,VCPU,Memory,Temporary Storage SSD(GB),Max data disks,Network bandwidth,Max cached IO throughput: IOPS/MBps (cache size in GiB),Max uncached IO throughput: IOPS/MBps
0,Standard_D1_v2,1,3.5,50,4,750,NaN,NaN
1,Standard_D2_v2,2,7.0,100,8,1500,NaN,NaN
2,Standard_D3_v2,4,14.0,200,16,3000,NaN,NaN
3,Standard_D4_v2,8,28.0,400,32,6000,NaN,NaN
4,Standard_D5_v2,16,56.0,800,64,12000,NaN,NaN
0,Standard_DS1_v2,1,3.5,7,4,750,4000/32 (43),3200/48
1,Standard_DS2_v2,2,7.0,14,8,1500,8000/64 (86),6400/96
2,Standard_DS3_v2,4,14.0,28,16,3000,16000/128 (172),12800/192
3,Standard_DS4_v2,8,28.0,56,32,6000,32000/256 (344),25600/384
4,Standard_DS5_v2,16,56.0,112,64,12000,64000/512 (688),51200/768


##  
<span  STYLE="color: black font-size: 15pt"> Loop through all VM Types (Compute Optimzed, General Purpose, Memory optimized, etc) and all VM Shapes using reference links </span>

In [88]:


def get_vmtypes_urls(vmtype_url):
      
        try:
             
            r = requests_retry_session().get(vmtype_url, timeout = 300)
        except Exception as e:
            
            vmtype_urls = []
            return vmtype_urls
        r_content = r.content
        soup = BeautifulSoup(r_content, "html.parser")
         
        vmtype_urls=[]
        vmtype_tuple=tuple()
    
    
        t=soup.find_all("table")[0]
        #print(t)
        for a in t.find_all('a',href=True,text=True):
             
            title=a.get("href")
            a_url=urljoin(vmtype_url,a.get('href'))
            #print(a.get("href"))
            #print(urljoin(vmtype_url,a.get('href')))
         
            if title not in ('sizes-gpu','sizes-hpc'):
                vmtype_tuple=(title, a_url)
                vmtype_urls.append(vmtype_tuple)
         
  
        return vmtype_urls    





def get_vmsize_urls(vmsizes_url):
      
        try:
             
            r = requests_retry_session().get(vmsizes_url, timeout = 300)
        except Exception as e:
            
            vmsize_urls = []
            return vmsize_urls
        r_content = r.content
        soup = BeautifulSoup(r_content, "html.parser")
         
        vmsize_urls=[]
        vmsize_tuple=tuple()
    
        for a in soup.find_all('a',href=True,text=True):
            if a.text.endswith('series'):
                title=a.get("href")
                a_url=urljoin(vmsizes_url,a.get('href'))
            
         
    
                vmsize_tuple=(title, a_url)
                vmsize_urls.append(vmsize_tuple)
         
  
        return vmsize_urls

In [89]:
get_vmtypes_urls('https://docs.microsoft.com/en-us/azure/virtual-machines/sizes')

[('sizes-general',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-general'),
 ('sizes-compute',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-compute'),
 ('sizes-memory',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-memory'),
 ('sizes-storage',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-storage')]

In [90]:
get_vmsize_urls('https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-general')

[('av2-series',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/av2-series'),
 ('dav4-dasv4-series',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/dav4-dasv4-series'),
 ('dv4-dsv4-series',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/dv4-dsv4-series'),
 ('ddv4-ddsv4-series',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/ddv4-ddsv4-series'),
 ('dv3-dsv3-series',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/dv3-dsv3-series'),
 ('ev3-esv3-series',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/ev3-esv3-series'),
 ('dv2-dsv2-series',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/dv2-dsv2-series'),
 ('dcv2-series',
  'https://docs.microsoft.com/en-us/azure/virtual-machines/dcv2-series')]

In [91]:

vmtype_list=get_vmtypes_urls('https://docs.microsoft.com/en-us/azure/virtual-machines/sizes')

df_concat=pd.DataFrame()
for vmtype,link in vmtype_list:
    print(vmtype,link)
    vmsize_list=get_vmsize_urls(link)
    for vmsize,link2 in vmsize_list:
        print(vmsize,link2)
        df=pd.DataFrame()
        
        df=specs_parse(link2)
        df['vm_type']=vmtype.split('-')[1].capitalize()
       # .apply(lambda x:x.capitalize())
       #df[cols].astype(str).apply(lambda x:x.split(' ')[0])
        df_concat=df_concat.append(df)
        
 

sizes-general https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-general
av2-series https://docs.microsoft.com/en-us/azure/virtual-machines/av2-series
dav4-dasv4-series https://docs.microsoft.com/en-us/azure/virtual-machines/dav4-dasv4-series
dv4-dsv4-series https://docs.microsoft.com/en-us/azure/virtual-machines/dv4-dsv4-series
ddv4-ddsv4-series https://docs.microsoft.com/en-us/azure/virtual-machines/ddv4-ddsv4-series
dv3-dsv3-series https://docs.microsoft.com/en-us/azure/virtual-machines/dv3-dsv3-series
ev3-esv3-series https://docs.microsoft.com/en-us/azure/virtual-machines/ev3-esv3-series
dv2-dsv2-series https://docs.microsoft.com/en-us/azure/virtual-machines/dv2-dsv2-series
dcv2-series https://docs.microsoft.com/en-us/azure/virtual-machines/dcv2-series
sizes-compute https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-compute
fsv2-series https://docs.microsoft.com/en-us/azure/virtual-machines/fsv2-series
sizes-memory https://docs.microsoft.com/en-us/azure/virtua

In [92]:
df_concat.head(10)

,Size,Memory,Temporary Storage SSD(GB),Max data disks,Max IO throughput,Network bandwidth,vm_type,VCPU,Max cached IO throughput: IOPS/MBps (cache size in GiB),Max uncached IO throughput: IOPS/MBps
0,Standard_A1_v2,2.0,10,2,2x500,250.0,General,NaN,NaN,NaN
1,Standard_A2_v2,4.0,20,4,4x500,500.0,General,NaN,NaN,NaN
2,Standard_A4_v2,8.0,40,8,8x500,1000.0,General,NaN,NaN,NaN
3,Standard_A8_v2,16.0,80,16,16x500,2000.0,General,NaN,NaN,NaN
4,Standard_A2m_v2,16.0,20,4,4x500,500.0,General,NaN,NaN,NaN
5,Standard_A4m_v2,32.0,40,8,8x500,1000.0,General,NaN,NaN,NaN
6,Standard_A8m_v2,64.0,80,16,16x500,2000.0,General,NaN,NaN,NaN
0,Standard_D2a_v4,8.0,50,4,NaN,1000.0,General,2.0,NaN,NaN
1,Standard_D4a_v4,16.0,100,8,NaN,2000.0,General,4.0,NaN,NaN
2,Standard_D8a_v4,32.0,200,16,NaN,4000.0,General,8.0,NaN,NaN


In [93]:
df_final=df_concat[['vm_type','Size','VCPU','Memory','Temporary Storage SSD(GB)','Max data disks','Max IO throughput','Max cached IO throughput: IOPS/MBps (cache size in GiB)', 'Max uncached IO throughput: IOPS/MBps','Network bandwidth']]
df_final.head(100)

,vm_type,Size,VCPU,Memory,Temporary Storage SSD(GB),Max data disks,Max IO throughput,Max cached IO throughput: IOPS/MBps (cache size in GiB),Max uncached IO throughput: IOPS/MBps,Network bandwidth
0,General,Standard_A1_v2,NaN,2.0,10,2,2x500,NaN,NaN,250.0
1,General,Standard_A2_v2,NaN,4.0,20,4,4x500,NaN,NaN,500.0
2,General,Standard_A4_v2,NaN,8.0,40,8,8x500,NaN,NaN,1000.0
3,General,Standard_A8_v2,NaN,16.0,80,16,16x500,NaN,NaN,2000.0
4,General,Standard_A2m_v2,NaN,16.0,20,4,4x500,NaN,NaN,500.0
...,...,...,...,...,...,...,...,...,...,...
2,General,Standard_DC4s_v2,4.0,112.0,200,4,NaN,8000/64,NaN,2.0
3,General,Standard_DC8_v2,8.0,168.0,400,8,NaN,16000/128,NaN,2.0
0,Compute,Standard_F2s_v2,2.0,4.0,16,4,NaN,4000/31 (32),3200/47,875.0
1,Compute,Standard_F4s_v2,4.0,8.0,32,8,NaN,8000/63 (64),6400/95,1750.0


##  
<span  STYLE="color: black font-size: 15pt"> Join the VM Size and Price dataframes. </span>

In [94]:
df_price=price[['name','regionName','linuxPrice']]  
df_price.columns = ['name', 'Price Region', 'Linux Price/hour']
#df_price['Linux Price/month']=df_price['Linux Price/hour'].map(lambda x: x * 24 * 31)

## To avoid SettingWithCopyWarning
pd.reset_option('mode.chained_assignment')
with pd.option_context('mode.chained_assignment', None):
    df_price.loc[:, 'Linux Price/month']=df_price['Linux Price/hour'].map(lambda x: x * 24 * 31)

In [95]:
#pd.merge(product,customer,on='Product_ID')
joined=pd.merge(df_final, df_price,how='left', left_on='Size', right_on='name')

In [96]:
joined

,vm_type,Size,VCPU,Memory,Temporary Storage SSD(GB),Max data disks,Max IO throughput,Max cached IO throughput: IOPS/MBps (cache size in GiB),Max uncached IO throughput: IOPS/MBps,Network bandwidth,name,Price Region,Linux Price/hour,Linux Price/month
0,General,Standard_A1_v2,NaN,2.0,10,2,2x500,NaN,NaN,250.0,Standard_A1_v2,West US,0.043,31.992
1,General,Standard_A2_v2,NaN,4.0,20,4,4x500,NaN,NaN,500.0,Standard_A2_v2,West US,0.091,67.704
2,General,Standard_A4_v2,NaN,8.0,40,8,8x500,NaN,NaN,1000.0,Standard_A4_v2,West US,0.191,142.104
3,General,Standard_A8_v2,NaN,16.0,80,16,16x500,NaN,NaN,2000.0,Standard_A8_v2,West US,0.400,297.600
4,General,Standard_A2m_v2,NaN,16.0,20,4,4x500,NaN,NaN,500.0,Standard_A2m_v2,West US,0.149,110.856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Storage,Standard_L16s_v2,16.0,128.0,NaN,32,NaN,NaN,16000/1280,6400.0,Standard_L16s_v2,West US,1.376,1023.744
203,Storage,Standard_L32s_v2,32.0,256.0,NaN,32,NaN,NaN,32000/1280,12800.0,Standard_L32s_v2,West US,2.752,2047.488
204,Storage,Standard_L48s_v2,48.0,384.0,NaN,32,NaN,NaN,48000/2000,NaN,Standard_L48s_v2,West US,4.128,3071.232
205,Storage,Standard_L64s_v2,64.0,512.0,NaN,32,NaN,NaN,64000/2000,NaN,Standard_L64s_v2,West US,5.504,4094.976


In [97]:
joined_selected=joined[['vm_type','Size','VCPU','Memory','Temporary Storage SSD(GB)','Max data disks','Max IO throughput','Max cached IO throughput: IOPS/MBps (cache size in GiB)','Max uncached IO throughput: IOPS/MBps','Network bandwidth','Price Region', 'Linux Price/hour','Linux Price/month']]

##  
<span  STYLE="color: black font-size: 15pt"> Output excel workbook with formats. </span>

In [99]:
# Write out to workbook with format

writer = pd.ExcelWriter("Azure_VM_Size_Price.xlsx", engine='xlsxwriter')
joined_selected.to_excel(writer,sheet_name = 'vmsize_price' )
workbook  = writer.book
worksheet = writer.sheets['vmsize_price']

#font_fmt = workbook.add_format({'font_name': 'Arial', 'font_size': 10})


header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': '#939DAD',
    'border': 1})

# Write the column headers with the defined format.
for col_num, value in enumerate(joined_selected.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)

    
    
    
# Add some cell formats.
 
format1= workbook.add_format({'num_format': '#,##0.000','border': 1})
format2= workbook.add_format({'font_name': 'Arial', 'font_size': 10,'border': 1})

# Set the column width and format.
worksheet.set_column('A:A', 5,format2 )
worksheet.set_column('B:B', 9,format2 )
worksheet.set_column('C:C', 16,format2 )
worksheet.set_column('D:D', 8,format2 )
worksheet.set_column('E:E', 9,format2 )
worksheet.set_column('F:F', 12,format2)
worksheet.set_column('G:G', 8,format2)
worksheet.set_column('H:H', 12,format2)
worksheet.set_column('I:I', 18,format2 )
worksheet.set_column('J:J', 16,format2 )
worksheet.set_column('K:K', 9,format2 )
worksheet.set_column('L:L', 12, format2)
worksheet.set_column('M:M', 12, format1)
worksheet.set_column('N:N', 13, format1)
writer.save() 
 